# Fine-tune a german GPT-2 model with 31 years of iX articles

In this Notebook I'm going to fine-tune a German GPT-2 from the [Huggingface model hub](https://huggingface.co/models). As fine-tune I using the iX archive (1988 - 2019) from [here](https://shop.heise.de/ix-archiv-1988-2019-usb-stick-1) and convert the PDF and HTML data into pure UTF-8 text with this [converter](https://github.com/rawar/ix-archive-utils).
The text file contains 31.304 german artcile pages.
The idea is that I use the iX articles to fine-tune an existing, german GPT-2 model to let me write iX articles like a pro. The original idea of this notebook are published by [Philipp Schmid](https://www.philschmid.de/fine-tune-a-non-english-gpt-2-model-with-huggingface). The original notebook is stored [here](https://colab.research.google.com/github/philschmid/fine-tune-GPT-2/blob/master/Fine_tune_a_non_English_GPT_2_Model_with_Huggingface.ipynb#scrollTo=PGEiQ1mhOyNv) in Google Colaboration.

## Install Jupyter Lab Extentions

In [1]:
!/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip

  Using cached pip-20.2.4-py2.py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2


## Install modules

In [2]:
!pip install -q transformers 

## GPU environment 
To check the GPU environment, I'm using the NVIDIA system management interface. 

In [3]:
!nvidia-smi

Sat Nov 14 20:53:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import time
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoModelWithLMHead
from transformers import pipeline

In [5]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 500)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")

In [7]:
model = AutoModelForCausalLM.from_pretrained("anonymous-german-nlp/german-gpt2")

In [8]:
configuration = model.config
print(configuration)

GPT2Config {
  "_name_or_path": "anonymous-german-nlp/german-gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 52000
}



In [9]:
data = pd.read_csv("./ix-archive.csv", sep='\n', header=None, encoding='utf-8')

In [11]:
data.head()

,0
0,"ix.0208.098-101 07.01.2008 15:30 Uhr Seite 100 REPORT Internet So ist beispielsweise die DSL-Flatrate eines T-DSL-Resale-Anbieters technisch nicht an den von ihm bereitgestellten DSL-Anschluss geknüpft. Gerade für kleinere DSL-Anbieter, die sich mit besonderen Features auf dem Markt positionieren, bringt dies entscheidende Vorteile. Sie können ihre Kunden unabhängig von deren Anschlussart versorgen. Wenig bekannt ist die Tatsache, dass der DSL-Anbieter nicht wissen muss, von welchem (Telefon..."
1,MARKT + TRENDS : Verschiedenes Verschiedenes Compaqs Alpha-Großrechner JavaScript-Bug im Navigator 4.x Elektronisches Papier marktreif NetCenter mit neuen Eigenschaften Symposium: Computer für alles und überall
2,"xx.1516.044-052.neu1.qxp 31.05.16 08:17 Seite 51 Im Normalfall geht man allerdings anders vor: Will man LibreOffice ausrollen und möchte hierzu ein bestimmtes Profil für die Nutzer haben, so erzeugt man zunächst eine saubere („clean“) Installation und lässt dann ein neues Profil (und somit auch eine initiale registrymodification.xcu) erzeugen. Nun korrigiert man in der Oberfläche alle Einstellungen, die später für alle Benutzer gelten sollen, beendet die Optionen und das Programm und hat nu..."
3,MARKT + TRENDS : Wirtschaft Wirtschaft DOMEA: Wahlkandidat für Behörden Aus USWeb und CKS wird Reinvent Helsinki Telephone steigt in Münster ein Siemens: SAP statt Peoplesoft Geht IBM-Chef zur Deutschen Bank? Kurz notiert
4,"ix.0810.096-100 05.07.2010 11:02 Uhr Seite 99 Routen häufiger in Benutzung sind als andere. Grundsätzlich können Datenpakete jeden möglichen Weg nehmen. Fallen Verbindungen aus oder sind überlastet, können die Router laufend Anpassungen vornehmen und Pakete umleiten. Dieses dynamische Routing kann Nachteile haben. Es ist allgemein bekannt, dass unverschlüsselte E-Mails wie eine Postkarte anzusehen sind, die jeder lesen kann, der Zugriff auf eines der Systeme hat, die die E-Mail transportiere..."


In [12]:
train, test = train_test_split(data,test_size=0.15)
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'
date_name = timestr = time.strftime("%Y%m%d-%H%M%S")
output_model_path = f"{date_name}_gpt2_ix"


In [13]:
train.to_csv(train_path, header=False, index=False)
test.to_csv(test_path, header=False, index=False)

In [14]:
print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 26345
Test dataset length: 4650


In [15]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128
    )

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

In [16]:
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/transformers/data/datasets/language_modeling.py:44: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


## Training arguments
A list of all training arguments can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). 

In [17]:
training_args = TrainingArguments(
    output_dir=output_model_path, 
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=50000, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [19]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  503616 KB |  503616 KB |  503616 KB |       0 B  |
|       from large pool |  490848 KB |  490848 KB |  490848 KB |       0 B  |
|       from small pool |   12768 KB |   12768 KB |   12768 KB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |  503616 KB |  503616 KB |  503616 KB |       0 B  |
|       from large pool |  490848 KB |  490848 KB |  490848 KB |

In [39]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


RuntimeError: CUDA out of memory. Tried to allocate 3.17 GiB (GPU 0; 15.78 GiB total capacity; 10.85 GiB already allocated; 924.00 MiB free; 13.79 GiB reserved in total by PyTorch)

In [32]:
trainer.save_model()

## Test the model
To test the model I'm going to use the [pipeline](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) object from the [transformers library](https://huggingface.co/transformers/index.html). With this API it is simple to generate articles texts like this.

In [33]:
ix_robo_author = pipeline(
    'text-generation',
    model=output_model_path, 
    tokenizer='anonymous-german-nlp/german-gpt2',
    config={'max_length':800}
)

In [34]:
ix_robo_author('Das neue iPhone 13')[0]['generated_text']

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Das neue iPhone 13 soll ab November erste 3,5""-Slots für mehr Speicher anbieten und unter anderem  eine 1,2-GHz-Xeon-CPU für 3,5""-Laufwerke besitzen. Das iPhone'

In [35]:
ix_robo_author('Die quelloffene Maschine-Learning-Bibliothek PyThorch ist')[0]['generated_text']

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Die quelloffene Maschine-Learning-Bibliothek PyThorch ist nicht nur ein ideales Hilfsmittel zur Verarbeitung agiler Prozesse. Angefangen beim einfachen Modellieren und Simulieren der Methoden und Methoden reichen deren Funktionalitäten bis hin zur Anwendung grafischer Benutzer'

In [36]:
ix_robo_author('Mit Hilfe von Apache Spark lassen sich')[0]['generated_text']

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Mit Hilfe von Apache Spark lassen sich alle Daten aus dem SAP-System gewinnen und auswerten, analysieren und auswerten. Eine Java- Engine kann aus bestehenden Diagrammen Daten an Spark übergeben, mit denen sie den Datenaustausch zwischen verschiedenen Diagram'

In [56]:
ix_robo_author('Apples neue Augmented Reallity Brille')[0]['generated_text']


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Apples neue Augmented Reallity Brille (SADT) bietet das Unternehmen einen  VR-Anpassungs- und Interaktions-Chip. Der soll die Entwicklung im Bereich der künstlichen neuronalen Netze, der kognitiven Neurobi'

In [38]:
ix_robo_author('Professionelle Softwareentwicklung benötigt heute')[0]['generated_text']

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Professionelle Softwareentwicklung benötigt heute nicht nur fundiertes Wissen über die Sprache selbst, sondern auch über die dahinter liegende Softwarekomponenten für das Testen eigener Prototypen. Eine Entwicklung unter Fremdbedingungen mag zwar in einer Softwarearchitektur resultieren, da Entwickler und Designer'

In [39]:
ix_robo_author('Der beste Editor für die Entwicklung von Python ist')[0]['generated_text']

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Der beste Editor für die Entwicklung von Python ist cut (siehe dazu Artikel ""Geisterstunde"" auf Seite 44, ""Online-Dinge""). Zwar haben Entwickler ihre eigenen Editoren, jedoch müssen sie sie nachrüsten. Daher'

In [51]:
ix_robo_author('Cloud Computing')[0]['generated_text']

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Cloud Computing: Mehr Sicherheit für Rechenzentren und Cloud Computing:  IDCs Studie IT-Sicherheit: Schwachstellenmanagement in verteilten Angriffen: DSGVO im  Rechenzentrum Sicherheitsvorfälle in Rechenzentren:  IDCs Umfrage IT'

In [54]:
ix_robo_author('Die Programmiersprache Java')[0]['generated_text']

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Die Programmiersprache Java hat sich in der Vergangenheit vielfach als De-facto-Standard gehalten und dient heute nicht nur als Referenz für Java-Applets wie Suns Java 1.9. In Java\xa02.0 sind die drei neuen Jini'